In [2]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

def get_residuals(actual_img, residual_img, wcs, apertures, filename):
    """
    Saves plots for each aperture to a single PDF file, including residual histograms.

    Parameters:
    - actual_img (numpy.ndarray): The original image array.
    - residual_img (numpy.ndarray): The residual image array.
    - wcs (WCS object): WCS object for the images.
    - apertures (list): List of aperture objects.
    - filename (str): Path to the output PDF file.
    """
    
    try:
        with PdfPages(filename) as pdf:
            for aperture in apertures:
                ap_pix = aperture.to_pixel(wcs)
                mask = ap_pix.to_mask(method='exact')

                mask_data = mask.get_values(actual_img)
                mask_res = mask.get_values(residual_img)

                noise_std = np.std(mask_res)

                cutout_data = mask.cutout(actual_img)
                cutout_res = mask.cutout(residual_img)

                fig, ax = plt.subplots(1, 3, figsize=(10, 5), dpi=300)
                ax[0].imshow(cutout_data, origin='lower')
                ax[0].set_title('Original')
                ax[1].imshow(cutout_res, origin='lower')
                ax[1].set_title('Residual')

                ax[2].hist(mask_res.flatten(), bins=100, range=(-8*noise_std, 8*noise_std),
                           align='left', color='k', label='Residual Image Values')
                ax[2].axvline(np.median(mask_data), color='r', label='Original Median')
                ax[2].axvline(np.median(mask_res), color='b', label='Residual Median')
                ax[2].legend(loc='upper left', fontsize=6)

                plt.tight_layout(rect=[0, 0, 0.75, 1])
                pdf.savefig(fig, bbox_inches='tight')
                plt.close(fig)

    except Exception as e:
        raise

In [10]:
from astropy.io import fits
from ds9parser import get_apertures
from astropy.wcs import WCS

file = fits.open('../images/epoch1/test.fits')
sci_img = file[1].data
sci_head = file[1].header
res_img = file[-1].data

aperatures = get_apertures('region.reg')
wcs_ = WCS(sci_head)

get_residuals(sci_img, res_img, wcs_, aperatures,'parseddata.pdf')

Set OBSGEO-B to    12.239720 from OBSGEO-[XYZ].
Set OBSGEO-H to 1226177244.676 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
